In [9]:
import pydicom as dcm
from pydicom.sequence import Sequence
from pydicom.dataset import Dataset
import zipfile
import os.path

ROOT = r"E:\Workspace\pymedphys"
ZIP_PATH = os.path.join(ROOT, "tests", "data", "dcmdose", "lfs-dicomorient-ct.zip")
EXTRACT_PATH = os.path.join(ROOT, "tests", "data", "dcmdose", "lfs-dicomorient-ct")
HFS_PLAN_PATH = os.path.join(EXTRACT_PATH, "HFS", "RP.DICOMORIENT.HFS.dcm")

# zipfile.ZipFile(ZIP_PATH, 'r').extractall(EXTRACT_PATH)

ds = dcm.dcmread(HFS_PLAN_PATH)
print(ds)

(0008, 0005) Specific Character Set              CS: 'ISO_IR 192'
(0008, 0012) Instance Creation Date              DA: '20181208'
(0008, 0013) Instance Creation Time              TM: '095655.191000'
(0008, 0016) SOP Class UID                       UI: RT Plan Storage
(0008, 0018) SOP Instance UID                    UI: 1.2.246.352.71.5.53598612033.417400.20181208095637
(0008, 0020) Study Date                          DA: '20181122'
(0008, 0030) Study Time                          TM: '111947'
(0008, 0050) Accession Number                    SH: '3558'
(0008, 0060) Modality                            CS: 'RTPLAN'
(0008, 0070) Manufacturer                        LO: 'Varian Medical Systems'
(0008, 0090) Referring Physician's Name          PN: ''
(0008, 1010) Station Name                        SH: 'HLT667S001'
(0008, 103e) Series Description                  LO: 'ARIA RadOnc Plans'
(0008, 1070) Operators' Name                     PN: 'mjenni03'
(0008, 1090) Manufacturer's Model Name     

In [10]:
type(ds.BeamSequence)

pydicom.sequence.Sequence

In [16]:
new_beam = Dataset()
new_beam.BeamNumber = 1 # Required, need to auto-set
new_beam.BeamName = "2x2 M, 3x3 J" # Optional, want to auto-set
new_beam.BeamDescription = "PyMedPhys: 2x2 M, 3x3 J" # Optional, want to auto-set
new_beam.BeamType = "STATIC"
new_beam.RadiationType = "PHOTON" # TODO: Handle electrons
# new_beam.PrimaryFluenceModeSequence = [Dataset] # Stated as optional but might need filling in
# new_beam.PrimaryFluenceModeSequence[0].FluenceMode = "STANDARD" # or "NON_STANDARD"?
# new_beam.PrimaryFluenceModeSequence[0].FluenceModeID needs filling in if above is NON_STANDARD
# new_beam.HighDoseTechniqueType = "HDR" # Might be needed for FFF?
new_beam.TreatmentMachineName = "" # Required but can leave empty

new_beam.PrimaryDosimeterUnit = "MU" # Optional

new_beam.BeamLimitingDeviceSequence = [Dataset(), Dataset()] #, Dataset(), Dataset()]
new_beam.BeamLimitingDeviceSequence[0].RTBeamLimitingDeviceType = "X" # or "ASYMX"
new_beam.BeamLimitingDeviceSequence[1].RTBeamLimitingDeviceType = "Y" # or "ASYMY"
# new_beam.BeamLimitingDeviceSequence[2].RTBeamLimitingDeviceType = "MLCX"
# new_beam.BeamLimitingDeviceSequence[3].RTBeamLimitingDeviceType = "MLCY"
new_beam.BeamLimitingDeviceSequence[0].NumberOfLeafJawPairs = 1
new_beam.BeamLimitingDeviceSequence[1].NumberOfLeafJawPairs = 1
# new_beam.BeamLimitingDeviceSequence[2].NumberOfLeafJawPairs = 1
# new_beam.BeamLimitingDeviceSequence[3].NumberOfLeafJawPairs = 1

new_beam.TreatmentDeliveryType = "TREATMENT" # Optional but might be best to set...

new_beam.NumberOfWedges = 0
# new_beam.WedgeSequence = [Dataset()]*NumberOfWedges
# new_beam.WedgeSequence[0].WedgeNumber = 1 # Unique within beam
# new_beam.WedgeSequence[0].WedgeType = "DYNAMIC" # or "STANDARD" or "MOTORIZED"
# new_beam.WedgeSequence[0].WedgeID = "" # Optional, probably not needed
# new_beam.WedgeSequence[0].AccessoryCode = "" # Optional but probably needed
# new_beam.WedgeSequence[0].WedgeAngle = 60
# new_beam.WedgeSequence[0].WedgeFactor = "" # Required but can leave empty
# new_beam.WedgeSequence[0].WedgeOrientation = 0 # Degree rel to Beam Limiting Device
new_beam.NumberOfCompensators = 0 # ASSUME NO COMPENSATORS
new_beam.NumberOfBoli = 0 # ASSUME NO BOLI
new_beam.NumberOfBlocks = 0 # ASSUME NO BLOCKS (TODO: check if used for electrons?)
# new_beam.ApplicatorSequence = [Dataset()] # Electrons & stereo
# new_beam.ApplicatorSequence[0].ApplicatorID = "<machine supplied ID>"
# new_beam.ApplicatorSequence[0].AccessoryCode = "" # Optional but may be needed
# new_beam.ApplicatorSequence[0].ApplicatorType = "ELECTRON_SQUARE" # many others possible
# new_beam.ApplicatorSequence[0].ApplicatorGeometrySequence = [Dataset()]
# new_beam.ApplicatorSequence[0].ApplicatorGeometrySequence[0].ApplicatorApertureShape = "SYM_SQUARE" # RECTANGLE and CIRCULAR also possible
# new_beam.ApplicatorSequence[0].ApplicatorGeometrySequence[0].ApplicatorOpening = 10 # Required for SQUARE and CIRCLE = length of side or diameter
# new_beam.ApplicatorSequence[0].ApplicatorGeometrySequence[0].ApplicatorOpeningX # Required if RECTANGLE
# new_beam.ApplicatorSequence[0].ApplicatorGeometrySequence[0].ApplicatorOpeningY # Required if RECTANGLE
# new_beam.ApplicatorSequence[0].ApplicatorDescription = "" # Optional but maybe want to define
# ASSUME NO ACCESSORIES? (what about cutouts?)

new_beam.FinalCumulativeMetersetWeight = 1.0

new_beam.NumberOfControlPoints = 2
new_beam.ControlPointSequence = [Dataset(), Dataset()]

#CP 1
new_beam.ControlPointSequence[0].ControlPointIndex = 0
new_beam.ControlPointSequence[0].CumulativeMetersetWeight = 0.0
new_beam.ControlPointSequence[0].NominalBeamEnergy = 0.0 # User supplied
new_beam.ControlPointSequence[0].DoseRateSet = 600.0 # User supplied

# new_beam.ControlPointSequence[0].WedgePositionSequence = [Dataset()]*NumberofWedges
# new_beam.ControlPointSequence[0].WedgePositionSequence[0].ReferencedWedgeNumber = 1
# new_beam.ControlPointSequence[0].WedgePositionSequence[0].WedgePosition = "IN"

new_beam.ControlPointSequence[0].BeamLimitingDevicePositionSequence = [Dataset(), Dataset(), Dataset(), Dataset()]
new_beam.ControlPointSequence[0].BeamLimitingDevicePositionSequence[0].RTBeamLimitingDeviceType = "X" # or "ASYMX"
new_beam.ControlPointSequence[0].BeamLimitingDevicePositionSequence[1].RTBeamLimitingDeviceType = "Y" # or "ASYMY"
# new_beam.ControlPointSequence[0].BeamLimitingDevicePositionSequence[2].RTBeamLimitingDeviceType = "MLCX"
# new_beam.ControlPointSequence[0].BeamLimitingDevicePositionSequence[3].RTBeamLimitingDeviceType = "MLCY"
new_beam.ControlPointSequence[0].BeamLimitingDevicePositionSequence[0].LeafJawPositions = [-30, 30]
new_beam.ControlPointSequence[0].BeamLimitingDevicePositionSequence[1].LeafJawPositions = [-30, 30]
# new_beam.ControlPointSequence[0].BeamLimitingDevicePositionSequence[2].LeafJawPositions = "TRICKSY MLC STUFF"
# new_beam.ControlPointSequence[0].BeamLimitingDevicePositionSequence[3].LeafJawPositions = "TRICKSY MLC STUFF"

new_beam.ControlPointSequence[0].GantryAngle = 0.0
new_beam.ControlPointSequence[0].GantryRotationDirection = "NONE"
new_beam.ControlPointSequence[0].BeamLimitingDeviceAngle = 0.0
new_beam.ControlPointSequence[0].BeamLimitingDeviceRotationDirection = "NONE"
new_beam.ControlPointSequence[0].PatientSupportAngle = 0.0
new_beam.ControlPointSequence[0].PatientSupportRotationDirection = "NONE"
new_beam.ControlPointSequence[0].TableTopEccentricAngle = 0.0
new_beam.ControlPointSequence[0].TableTopEccentricRotationDirection = "NONE"
new_beam.ControlPointSequence[0].TableTopPitchAngle = 0.0
new_beam.ControlPointSequence[0].TableTopPitchRotationDirection = "NONE"
new_beam.ControlPointSequence[0].TableTopRollAngle = 0.0
new_beam.ControlPointSequence[0].TableTopRollRotationDirection = "NONE"
new_beam.ControlPointSequence[0].TableTopVerticalPosition = "" # Required but can be empty
new_beam.ControlPointSequence[0].TableTopLongitudinalPosition = "" # Required but can be empty
new_beam.ControlPointSequence[0].TableTopLateralPosition = "" # Required but can be empty
new_beam.ControlPointSequence[0].IsocenterPosition = "" # Required but can be empty
new_beam.ControlPointSequence[0].SourceToSurfaceDistance = 900 # Optional but we should fill

#CP2
new_beam.ControlPointSequence[1].ControlPointIndex = 1
new_beam.ControlPointSequence[1].CumulativeMetersetWeight = 1.0


(300a, 00b2) Treatment Machine Name              SH: ''
(300a, 00b3) Primary Dosimeter Unit              CS: 'MU'
(300a, 00b6)  Beam Limiting Device Sequence   2 item(s) ---- 
   (300a, 00b8) RT Beam Limiting Device Type        CS: 'X'
   (300a, 00bc) Number of Leaf/Jaw Pairs            IS: "1"
   ---------
   (300a, 00b8) RT Beam Limiting Device Type        CS: 'Y'
   (300a, 00bc) Number of Leaf/Jaw Pairs            IS: "1"
   ---------
(300a, 00c0) Beam Number                         IS: "1"
(300a, 00c2) Beam Name                           LO: '2x2 M, 3x3 J'
(300a, 00c3) Beam Description                    ST: 'PyMedPhys: 2x2 M, 3x3 J'
(300a, 00c4) Beam Type                           CS: 'STATIC'
(300a, 00c6) Radiation Type                      CS: 'PHOTON'
(300a, 00ce) Treatment Delivery Type             CS: 'TREATMENT'
(300a, 00d0) Number of Wedges                    IS: "0"
(300a, 00e0) Number of Compensators              IS: "0"
(300a, 00ed) Number of Boli                      IS